In [ ]:
import tensorflow as tf
tf.device("gpu:5")

In [2]:
# -*- coding: utf-8 -*-
'''ResNet50 model for Keras.
# Reference:
- [Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385)
Adapted from code contributed by BigMoyan.
https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py
'''
from __future__ import print_function

import numpy as np
import warnings

from tensorflow.keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras.layers import (Dense
                                    ,Activation
                                    ,Flatten
                                    ,Conv2D
                                    ,MaxPooling2D
                                    ,GlobalMaxPooling2D
                                    ,ZeroPadding2D
                                    ,AveragePooling2D
                                    ,GlobalAveragePooling2D
                                    ,BatchNormalization
                                    ,concatenate)
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from tensorflow.python.keras.utils import layer_utils
from tensorflow.python.keras.utils.data_utils import get_file
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from tensorflow.keras.applications.imagenet_utils import preprocess_input
from keras_applications.imagenet_utils import _obtain_input_shape
from tensorflow.keras.utils import get_source_inputs


WEIGHTS_PATH = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels.h5'
WEIGHTS_PATH_NO_TOP = 'https://github.com/fchollet/deep-learning-models/releases/download/v0.2/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'


def identity_block(input_tensor, kernel_size, filters, stage, block, layer_num=""):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = layer_num + 'res' + str(stage) + block + '_branch'
    bn_name_base = layer_num + 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2), layer_num=""):
    """conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = layer_num + 'res' + str(stage) + block + '_branch'
    bn_name_base = layer_num + 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x


def conv_block2(input_tensor, kernel_size, filters, stage, block, strides=(1, 1), dilation_rate=(2, 2), layer_num=""):
    """conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with strides=(2,2)
    And the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = layer_num + 'res' + str(stage) + block + '_branch'
    bn_name_base = layer_num + 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), strides=strides, dilation_rate=dilation_rate,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same', dilation_rate=dilation_rate,
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), dilation_rate=dilation_rate, name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides, dilation_rate=dilation_rate,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = Activation('relu')(x)
    return x
    
    
def channel_attention_block(input_tensor, block, reduction_ratio=16, layer_num=""):
    initial_input = input_tensor
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channels = initial_input.shape[channel_axis]
    attention_shape = (1, 1, channels)
    
    x = GlobalAveragePooling2D()(input_tensor)
    x = Dense(units=channels//reduction_ratio, activation='relu', name=layer_num + "squeeze" + block )(x)
    x = Dense(units=channels, activation='sigmoid', name=layer_num + "excitation" + block )(x)
    
    if K.image_data_format() == 'channels_first':
        x = layers.Permute((3, 1, 2))(x)
            
    x = layers.multiply([initial_input, x])
    return layers.add([initial_input, x])
    


def ResNet50(require_flatten=True, weights='imagenet',
             input_tensor=None, input_shape=None,
             pooling=None,
             classes=1000):
    """Instantiates the ResNet50 architecture.
    Optionally loads weights pre-trained
    on ImageNet. Note that when using TensorFlow,
    for best performance you should set
    `image_data_format="channels_last"` in your Keras config
    at ~/.keras/keras.json.
    The model and the weights are compatible with both
    TensorFlow and Theano. The data format
    convention used by the model is the one
    specified in your Keras config file.
    # Arguments
        require_flatten: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization)
            or "imagenet" (pre-training on ImageNet).
        input_tensor: optional Keras tensor (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `require_flatten` is False (otherwise the input shape
            has to be `(224, 224, 3)` (with `channels_last` data format)
            or `(3, 224, 244)` (with `channels_first` data format).
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 197.
            E.g. `(200, 200, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `require_flatten` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `require_flatten` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and require_flatten and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `require_flatten`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=224,
                                      min_size=197,
                                      data_format=K.image_data_format(),
                                      require_flatten=require_flatten)

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1')(x)
    x = BatchNormalization(axis=bn_axis, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling2D((7, 7), name='avg_pool')(x)

    if require_flatten:
        x = Flatten()(x)
        x = Dense(classes, activation='softmax', name='fc1000')(x)
    else:
        if pooling == 'avg':
            x = GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = get_source_inputs(input_tensor)
    else:
        inputs = img_input
    # Create model.
    model = Model(inputs, x, name='resnet50')

    # load weights
    if weights == 'imagenet':
        if require_flatten:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models',
                                    md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
        else:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models',
                                    md5_hash='a268eb855778b3df3c7506639542a6af')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if require_flatten:
                maxpool = model.get_layer(name='avg_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1000')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model


def MultiView_MMG_DensityDC_CA(require_flatten=True, weights='imagenet',
             input_tensor1=None, input_shape1=None,
             input_tensor2=None, input_shape2=None,
             pooling=None,
             classes=4):
    """
    """
    def base_network(img_input, layer_num=""):
        layer_num = str(layer_num)
        x = ZeroPadding2D((3, 3))(img_input)
        x = Conv2D(64, (7, 7), strides=(2, 2), name='conv1'+"_"+layer_num)(x)
        x = channel_attention_block(x, block="1", layer_num=layer_num)
        x = BatchNormalization(axis=bn_axis, name='bn_conv1'+"_"+layer_num)(x)
        x = Activation('relu')(x)
        x = MaxPooling2D((2, 2), strides=(2, 2))(x)

        x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1), layer_num=layer_num)
        x = channel_attention_block(x, block="2", layer_num=layer_num)
        x = identity_block(x, 3, [64, 64, 256], stage=2, block='b', layer_num=layer_num)
        x = identity_block(x, 3, [64, 64, 256], stage=2, block='c', layer_num=layer_num)

        x = conv_block(x, 3, [128, 128, 512], stage=3, block='a', layer_num=layer_num)
        x = channel_attention_block(x, block="3", layer_num=layer_num)
        x = identity_block(x, 3, [128, 128, 512], stage=3, block='b', layer_num=layer_num)
        x = identity_block(x, 3, [128, 128, 512], stage=3, block='c', layer_num=layer_num)
        x = identity_block(x, 3, [128, 128, 512], stage=3, block='d', layer_num=layer_num)

        x = conv_block2(x, 3, [256, 256, 1024], stage=4, block='a', dilation_rate=(2, 2), layer_num=layer_num)
        x = channel_attention_block(x, block="4", layer_num=layer_num)
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b', layer_num=layer_num)
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c', layer_num=layer_num)
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d', layer_num=layer_num)
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e', layer_num=layer_num)
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f', layer_num=layer_num)

        x = conv_block2(x, 3, [512, 512, 2048], stage=5, block='a', dilation_rate=(4, 4), layer_num=layer_num)
        x = channel_attention_block(x, block="5", layer_num=layer_num)
        x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b', layer_num=layer_num)
        x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c', layer_num=layer_num)

        # x = AveragePooling2D((7, 7), name='avg_pool')(x)
        x = GlobalAveragePooling2D()(x)
        
        return x
    

    def img_input(input_shape, input_tensor=None):
        if input_tensor is None:
            img_input = Input(shape=input_shape)
        else:
            if not K.is_keras_tensor(input_tensor):
                img_input = Input(tensor=input_tensor, shape=input_shape)
            else:
                img_input = input_tensor
        return img_input
    
    
    if weights not in {'imagenet', None}:
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization) or `imagenet` '
                         '(pre-training on ImageNet).')

    if weights == 'imagenet' and require_flatten and classes != 1000:
        raise ValueError('If using `weights` as imagenet with `require_flatten`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    input_shape1 = _obtain_input_shape(input_shape1,
                                      default_size=224,
                                      min_size=197,
                                      data_format=K.image_data_format(),
                                      require_flatten=require_flatten)
    
    input_shape2 = _obtain_input_shape(input_shape2,
                                      default_size=224,
                                      min_size=197,
                                      data_format=K.image_data_format(),
                                      require_flatten=require_flatten)
    
    img_input1 = img_input(input_shape1, input_tensor1)
    img_input2 = img_input(input_shape2, input_tensor2)
            
            
    if K.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1

    x1 = base_network(img_input1, 1)
    x2 = base_network(img_input2, 2)
    # combine multiple n/w branches
    x = concatenate([x1, x2])
    x = Dense(2048, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(classes, activation='softmax', name='predictions')(x)


    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor1 is not None and input_tensor2 is not None:
        inputs = (get_source_inputs(input_tensor1), get_source_inputs(input_tensor2))
    else:
        inputs = (img_input1, img_input2)
    # Create model.
    model = Model(inputs, x, name='Multi-View-MMG-Density-DC-CA')

    # load weights
    if weights == 'imagenet':
        if require_flatten:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels.h5',
                                    WEIGHTS_PATH,
                                    cache_subdir='models',
                                    md5_hash='a7b3fe01876f51b976af0dea6bc144eb')
        else:
            weights_path = get_file('resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5',
                                    WEIGHTS_PATH_NO_TOP,
                                    cache_subdir='models',
                                    md5_hash='a268eb855778b3df3c7506639542a6af')
        model.load_weights(weights_path)
        if K.backend() == 'theano':
            layer_utils.convert_all_kernels_in_model(model)

        if K.image_data_format() == 'channels_first':
            if require_flatten:
                maxpool = model.get_layer(name='avg_pool')
                shape = maxpool.output_shape[1:]
                dense = model.get_layer(name='fc1000')
                layer_utils.convert_dense_weights_data_format(dense, shape, 'channels_first')

            if K.backend() == 'tensorflow':
                warnings.warn('You are using the TensorFlow backend, yet you '
                              'are using the Theano '
                              'image data format convention '
                              '(`image_data_format="channels_first"`). '
                              'For best performance, set '
                              '`image_data_format="channels_last"` in '
                              'your Keras config '
                              'at ~/.keras/keras.json.')
    return model

if __name__ == '__main__':
#     model = ResNet50(require_flatten=True, weights='imagenet')
    model = ResNet50(require_flatten=True, weights=None)

#     img_path = 'elephant.jpg'
#     img = image.load_img(img_path, target_size=(224, 224))
#     x = image.img_to_array(img)
#     x = np.expand_dims(x, axis=0)
#     x = preprocess_input(x)
#     print('Input image shape:', x.shape)

#     preds = model.predict(x)
#     print('Predicted:', decode_predictions(preds))

In [3]:
base_model = ResNet50(require_flatten=True, weights="imagenet")
new_model = MultiView_MMG_DensityDC_CA(require_flatten=True, weights=None)

In [4]:
# load pretrained weights to the new model.
base_model.trainable = True
layers = [l.name for l in base_model.layers]
new_modellayers = [l.name for l in new_model.layers]
len_layers = len(layers)
new_modellen_layers = len(new_modellayers)
print(len_layers)
print(new_modellen_layers)
# set pre-trained weight
count = 0
for i, layer_base in enumerate(base_model.layers):
    if i < len_layers-1:
        for layer_loaded in new_model.layers:
            base_layer = layer_base.name
            loaded_layer = layer_loaded.name
            # match layername and copy weights from pretrained base model
            if loaded_layer[0] in ["1", "2"]:
                loaded_layer = loaded_layer[1:]
                
            if base_layer == loaded_layer:
                try:
                    layer_base.set_weights(layer_loaded.get_weights())
                    count += 1
                except Exception as e:
                    print(e)
                    
print(f"loaded weight for {count} layers.")

177
404
loaded weight for 208 layers.


In [5]:
import tensorflow
tensorflow.keras.utils.plot_model(new_model, "{}.png".format("Multi-View-MMG-Density-DC-CA-v0.1"), show_shapes=True)

In [6]:
model_json_path = 'Multi-View-MMG-Density-DC-CA-v0.1.json'

json_string = new_model.to_json()

with open(model_json_path, 'w') as f:
    f.write(json_string)

new_model.save_weights('Multi-View-MMG-Density-DC-CA-v0.1.h5')

/opt/anaconda3/envs/submarine_clearml/lib/python3.8/site-packages/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [7]:
# model = ResNet50(require_flatten=True, weights=None)

img_path = '/home/perceptra/submarine-project-code/amir/breast_density/model_dev/elephant.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)
print('Input image shape:', x.shape)

preds = new_model.predict((x, x))
print('Predicted:', preds)

Input image shape: (1, 224, 224, 3)
Predicted: [[2.6879831e-25 1.0000000e+00 5.4768410e-20 7.6911307e-26]]
